In [1]:
import pyodbc
import pandas as pd

In [2]:
pd.reset_option("display.width", None)
pd.set_option("display.expand_frame_repr", None)

In [6]:
driver = '{SQL Server}'
server = 'DESKTOP-41IVAA9'
database = 'CUSTOMERS_TEST'
username = 'sa'
password = '1'
connection_string = f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}"

In [13]:
connection = pyodbc.connect(connection_string)
cursor = connection.cursor()

In [8]:
def sorgu(query, select=True):
    cursor = connection.cursor()
    cursor.execute(query)
    if select:
        rows = cursor.fetchall()
        rows = [list(row) for row in rows]
        columns = [column[0] for column in cursor.description]
        return pd.DataFrame(rows, columns=columns)

In [ ]:
df = sorgu("SELECT * FROM CUSTOMERS")
df

#### SORU 6

In [ ]:
# -- SORU 6
df = sorgu("SELECT * FROM CUSTOMERS WHERE NAMESURNAME LIKE 'A%'")
df


#### SORU 7

In [ ]:
# -- SORU 7
df = sorgu("""
SELECT * 
FROM CUSTOMERS 
WHERE NAMESURNAME LIKE 'A%'
AND GENDER = 'E'""")
df


#### SORU 8

In [ ]:
# -- SORU 8
df = sorgu("""
SELECT *
FROM CUSTOMERS
WHERE BIRTHDATE BETWEEN '1990-01-01' AND '1995-12-31'
""")
df


In [ ]:
# -- SORU 8
df = sorgu("""
SELECT *
FROM CUSTOMERS
WHERE YEAR(BIRTHDATE) BETWEEN 1990 AND 1995
""")
df


#### SORU 9

In [ ]:
# -- SORU 9
df = sorgu("""
SELECT *
FROM CUSTOMERS AS C
JOIN CITIES AS CT ON CT.ID=C.CITYID
WHERE CT.CITY='İSTANBUL'
""")
df


#### SORU 10

In [ ]:
# -- SORU 10
df = sorgu("""
SELECT *
FROM CUSTOMERS
WHERE CITYID IN (
    SELECT ID
    FROM CITIES
    WHERE CITY='İSTANBUL')
""")
df


#### SORU 11

In [ ]:
# -- SORU 11
df = sorgu("""
SELECT 
    CT.CITY AS SEHIR,
    COUNT(*) AS [KISI SAYISI]
FROM CUSTOMERS AS C
RIGHT JOIN CITIES AS CT ON CT.ID=C.CITYID
GROUP BY CT.CITY
ORDER BY [KISI SAYISI] DESC
""")
df


In [ ]:
# -- SORU 11
df = sorgu("""
SELECT 
    *,
    (SELECT COUNT(*)
    FROM CUSTOMERS
    WHERE CUSTOMERS.CITYID=CITIES.ID) AS [SAYI]
FROM CITIES
ORDER BY SAYI DESC
""")
df


#### SORU 12

In [ ]:
df = sorgu("""
SELECT
    CT.CITY,
    COUNT(*) AS [KULLANICI SAYISI]
FROM CUSTOMERS
JOIN CITIES AS CT ON CT.ID=CUSTOMERS.CITYID
GROUP BY CT.CITY
HAVING COUNT(*)>10
ORDER BY [KULLANICI SAYISI] DESC
""")
df


In [ ]:
df = sorgu("""
SELECT
    (SELECT CITY FROM CITIES WHERE ID=CUSTOMERS.CITYID),
    COUNT(*) AS [KULLANICI SAYISI]
FROM CUSTOMERS
GROUP BY CUSTOMERS.CITYID
HAVING COUNT(*)>10
ORDER BY [KULLANICI SAYISI] DESC
""")
df


#### SORU 13

In [ ]:
df = sorgu("""
SELECT
    CT.CITY,
    GENDER, 
    COUNT(*) AS [KİŞİ SAYISI]
FROM CUSTOMERS
JOIN CITIES AS CT ON CT.ID=CITYID
GROUP BY 
    CT.CITY,
    GENDER
ORDER BY 
    1 ASC,
    3 DESC
""")
df


#### SORU 14

In [ ]:
df = sorgu("""
SELECT
    CITY,
    (SELECT COUNT(*) FROM CUSTOMERS WHERE CITYID=C.CITYID) AS [MÜŞTERİ SAYISI],
    (SELECT COUNT(*) FROM CUSTOMERS WHERE GENDER='K' AND CITYID=C.CITYID) AS [KADIN SAYISI],
    (SELECT COUNT(*) FROM CUSTOMERS WHERE GENDER='E' AND CITYID=C.CITYID)  AS [ERKEK SAYISI]
FROM CUSTOMERS AS C
JOIN CITIES AS CT ON CT.ID=C.CITYID
GROUP BY CITYID, CITY
ORDER BY 2 DESC
""")
df


In [ ]:
df = sorgu("""
SELECT 
    CITY,
    (SELECT COUNT(*) FROM CUSTOMERS WHERE CT.ID=CITYID) AS [MÜŞTERİ SAYISI],
    (SELECT COUNT(*) FROM CUSTOMERS WHERE GENDER='K' AND CT.ID=CITYID) AS [KADIN SAYISI],
    (SELECT COUNT(*) FROM CUSTOMERS WHERE GENDER='E' AND CT.ID=CITYID)  AS [ERKEK SAYISI]
FROM CITIES AS CT
ORDER BY 2 DESC
""")
df


#### SORU 15

In [ ]:
df = sorgu("""
ALTER TABLE CUSTOMERS 
ADD AGEGROUP VARCHAR(50);
""")
df


#### SORU 16

In [ ]:
df = sorgu("""
UPDATE CUSTOMERS 
SET AGEGROUP='20-35 YAŞ' 
WHERE DATEDIFF(YEAR, BIRTHDATE, GETDATE()) BETWEEN 20 AND 35

UPDATE CUSTOMERS 
SET AGEGROUP='36-45 YAŞ' 
WHERE DATEDIFF(YEAR, BIRTHDATE, GETDATE()) BETWEEN 36 AND 45

UPDATE CUSTOMERS 
SET AGEGROUP='46-55 YAŞ' 
WHERE DATEDIFF(YEAR, BIRTHDATE, GETDATE()) BETWEEN 46 AND 55

UPDATE CUSTOMERS 
SET AGEGROUP='56-65 YAŞ' 
WHERE DATEDIFF(YEAR, BIRTHDATE, GETDATE()) BETWEEN 56 AND 65

UPDATE CUSTOMERS 
SET AGEGROUP='65+ YAŞ' 
WHERE DATEDIFF(YEAR, BIRTHDATE, GETDATE()) > 65
""", select=False)
df


In [ ]:
df = sorgu("""
SELECT *
FROM CUSTOMERS
WHERE AGEGROUP IS NULL
""")
df


#### SORU 17

In [ ]:
df = sorgu("""
SELECT 
    ARALIK,
    COUNT(*) AS SAYI
FROM (
    SELECT 
        CASE
            WHEN DATEDIFF(YEAR, BIRTHDATE, GETDATE()) BETWEEN 20 AND 35 THEN '20-35 YAŞ'
            WHEN DATEDIFF(YEAR, BIRTHDATE, GETDATE()) BETWEEN 36 AND 45 THEN '36-45 YAŞ'
            WHEN DATEDIFF(YEAR, BIRTHDATE, GETDATE()) BETWEEN 46 AND 55 THEN '46-55 YAŞ'
            WHEN DATEDIFF(YEAR, BIRTHDATE, GETDATE()) BETWEEN 56 AND 65 THEN '56-65 YAŞ'
            WHEN DATEDIFF(YEAR, BIRTHDATE, GETDATE()) > 65 THEN '65+ YAŞ' 
        END AS ARALIK,
        *
    FROM CUSTOMERS) AS TEMP
GROUP BY ARALIK
""")
df


#### SORU 18

In [ ]:
df = sorgu("""
SELECT *
FROM CUSTOMERS AS C
JOIN CITIES AS CT ON CT.ID=C.CITYID
JOIN DISTRICTS AS D ON D.ID=C.DISTRICT
WHERE 
    CT.CITY='İSTANBUL'
    AND D.DISTRICT<>'KADIKÖY'
""")
df


#### SORU 19

In [ ]:
"""
DELETE FROM CITIES WHERE CITY='ANKARA'
DELETE FROM CITIES WHERE CITY='İSTANBUL'
"""

"""
SELECT *
FROM CITIES
"""

df = sorgu("""
SELECT *
FROM CUSTOMERS AS C
LEFT JOIN CITIES AS CT ON CT.ID=C.CITYID
WHERE CT.CITY IS NULL
""")
df


#### SORU 20

In [ ]:
df = sorgu("""
SET IDENTITY_INSERT CITIES ON
INSERT INTO CITIES (ID, CITY) VALUES (6, 'ANKARA')
INSERT INTO CITIES (ID, CITY) VALEUS (34, 'İSTANBUL')
""")
df


#### SORU 21

In [ ]:
df = sorgu("""
SELECT 
    *,
    LEFT(TELNR1, 5) AS OPERATOR1,
    SUBSTRING(TELNR2, 2, 3) AS OPERATOR2
FROM CUSTOMERS
""")
df


#### SORU 22

In [ ]:
df = sorgu("""
SELECT 
	XOPS1+XOPS2 AS XOPS,
	YOPS1+YOPS2 AS YOPS,
	ZOPS1+ZOPS2 AS ZOPS
FROM (
	SELECT 
		SUM(CASE 
			WHEN SUBSTRING(TELNR1, 2, 2)='50' OR SUBSTRING(TELNR1, 2, 2)='55' THEN 1 
			ELSE 0
		END) AS XOPS1,
		SUM(CASE 
			WHEN SUBSTRING(TELNR1, 2, 2)='54'THEN 1 
			ELSE 0
		END) AS YOPS1,
		SUM(CASE 
			WHEN SUBSTRING(TELNR1, 2, 2)='53'THEN 1 
			ELSE 0
		END) AS ZOPS1,
		SUM(CASE 
			WHEN SUBSTRING(TELNR2, 2, 2)='50' OR SUBSTRING(TELNR2, 2, 2)='55' THEN 1 
			ELSE 0
		END) AS XOPS2,
		SUM(CASE 
			WHEN SUBSTRING(TELNR2, 2, 2)='54'THEN 1 
			ELSE 0
		END) AS YOPS2,
		SUM(CASE 
			WHEN SUBSTRING(TELNR2, 2, 2)='53'THEN 1 
			ELSE 0
		END) AS ZOPS2
	FROM CUSTOMERS) AS TEMP
    """)
df

#### SORU 23

In [10]:
df = sorgu("""
SELECT 
	CT.CITY,
	D.DISTRICT,
	COUNT(C.ID) AS [KİSİ SAYISI]
FROM CUSTOMERS AS C
JOIN CITIES AS CT ON CT.ID=C.CITYID
JOIN DISTRICTS AS D ON D.ID=C.DISTRICTID
GROUP BY
	CT.CITY,
	D.DISTRICT
ORDER BY
	1,
	3 DESC
    """)
df

,CITY,DISTRICT,KİSİ SAYISI
0,ADANA,SEYHAN,8
1,ADANA,ALADAĞ,6
2,ADANA,YÜREĞİR,2
3,ADIYAMAN,ADIYAMAN MERKEZ,5
4,ADIYAMAN,BESNİ,2
...,...,...,...
390,ZONGULDAK,ALAPLI,2
391,ZONGULDAK,ÇAYCUMA,2
392,ZONGULDAK,DEVREK,2
393,ZONGULDAK,GÖKÇEBEY,1


#### SORU 24

In [15]:
cursor.execute("SET LANGUAGE Turkish")
df = sorgu("""
SELECT 
	NAMESURNAME,
	DATENAME(dw, BIRTHDATE) AS [DOĞUM GÜNÜ]
FROM CUSTOMERS
    """)
df

,NAMESURNAME,DOĞUM GÜNÜ
0,Sevda AKÇAN,Salı
1,Sebahat ŞERALI,Cuma
2,Irmak HAMİDİ,Salı
3,Tuğçe AKKOÇ,Pazar
4,Necdet ERÇAM,Perşembe
...,...,...
946,Nisa nur MÜFTÜLÜĞÜ,Pazartesi
947,Yusuf Eymen TARI,Cumartesi
948,Esma nur HIZLAR,Perşembe
949,Şaziye AKKOCA,Pazar


#### SORU 25

In [17]:
df = sorgu("""
SELECT 
	NAMESURNAME,
	BIRTHDATE
FROM CUSTOMERS
WHERE 
	DAY(BIRTHDATE)=DAY(GETDATE())
	AND DATEPART(MONTH, BIRTHDATE)=DATEPART(MONTH, GETDATE())
    """)
df

,NAMESURNAME,BIRTHDATE
0,Muhammed Mustafa CİMEN,1965-07-23
1,Ömer SOFT,1941-07-23
2,Çiğdem SEVENCAN,1948-07-23
